## Import test data with data generator

In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import os 

In [2]:
TEST_DIR = r"\\192.168.100.160\CloudSource\CloudSource_share\tharhtetsan\open_pose\ths_training\sit_stand_dataset\test" # path to test folder

In [3]:
categories = ['Negative','Sit','Stand']
batch_size = 1
IMAGE_SIZE = (224, 128)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
                                TEST_DIR,
                                target_size=IMAGE_SIZE,
                                color_mode="rgb",
                                shuffle = False,
                                class_mode='categorical',
                                batch_size=batch_size)

Found 7621 images belonging to 3 classes.


In [4]:
import numpy as np
test_steps_per_epoch = np.math.ceil(test_generator.samples / test_generator.batch_size)

#Tensorflow model accuracy

In [5]:
tf_model = load_model(r'C:\Users\Clsm\Desktop\test_files\cnn_model_with_custom_gen_conv_layer_4.h5')

In [11]:
import numpy as np
class_label = ['negative','sit','stand']
true_labels = []
predicted_labels = []

import matplotlib.pyplot as plt

count = 0
for image,label in test_generator:
    
    
    
    predicted_temp = tf_model.predict(image)
    for i in range(len(label)):
        true_labels.append(class_label[np.argmax(label[i])])
        predicted_labels.append(class_label[np.argmax(predicted_temp[i])])
        

    count = count+1
    if count > 1000:
        break
    
test_accuracy = 100 * np.sum(np.array(predicted_labels)==true_labels) / len(predicted_labels)
print("TF model Test accuracy : {}".format(test_accuracy))

TF model Test accuracy : 96.2037962037962


## Tflite model accuracy

In [7]:
# Convert the model.
model = load_model(r'C:\Users\Clsm\Desktop\test_files\cnn_model_with_custom_gen_conv_layer_4.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('sit_stand_cnn.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Clsm\AppData\Local\Temp\tmpw51c03s3\assets


In [8]:
# Load TFLite model and see some details about input/output
import tensorflow as tf
tflite_interpreter = tf.lite.Interpreter(model_path=r"sit_stand_cnn.tflite")
tflite_interpreter.allocate_tensors()

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()


print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

== Input details ==
name: serving_default_conv2d_input:0
shape: [  1 224 128   3]
type: <class 'numpy.float32'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [1 3]
type: <class 'numpy.float32'>


In [10]:
import numpy as np
true_labels = []
predicted_labels = []
class_label = ['negative','sit','stand']

count = 0
for image,label in test_generator:
    
    
    

    for i in range(len(label)):
        current_image = image[i]
        #print(current_image.shape)
        current_image = np.array([current_image])
        #print(current_image.shape)
        
        # Run inference
        tflite_interpreter.set_tensor(input_details[0]['index'], current_image)
        tflite_interpreter.invoke()
        tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
        #print("Prediction results shape:", tflite_model_predictions.shape)
       
        true_labels.append(categories[np.argmax(label[i])])
        predicted_labels.append(categories[np.argmax(tflite_model_predictions[0])])
        

    count = count+1
    if count > 1000:
        break

test_accuracy = 100 * np.sum(np.array(predicted_labels)==true_labels) / len(predicted_labels)
print("TF lite model Test accuracy : {}".format(test_accuracy))

TF lite model Test accuracy : 95.2047952047952
